In [ ]:
%pip install --upgrade pip

In [ ]:
%pip install numpy

In [ ]:
%pip install matplotlib

In [ ]:
%pip install geopandas

In [ ]:
%pip install pandas

In [ ]:
%pip install fiona

In [ ]:
%pip install shapely

In [ ]:
%pip install pyproj

In [ ]:
import re
import collections
import difflib
import unicodedata
import warnings

import numpy as np

from matplotlib.patches import Polygon as mplPolygon
import matplotlib.pyplot as mplPlt
from matplotlib.collections import PatchCollection as mplPC
import matplotlib.colors as mplColors
import matplotlib.animation as mplAnimation

import geopandas as gpd
import pandas as pd
import fiona

from shapely.geometry import MultiPolygon, box
from shapely.ops import transform

from pyproj import Transformer

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning, module="shapely")

In [ ]:
# ------------------------------------------------------------
# 1. Load CSV with organization counts
# ------------------------------------------------------------
csv_path = 'Datasets/quantum_computing_report___organizations.csv'

df = pd.read_csv(csv_path,sep=';',usecols=["Organization","Acronym","Type","Link","Location","Specialization"])
df['Location2']=''
df = df.reset_index()

for index, row in df.iterrows():
    if ", " in row['Location']:
        aux = row['Location'].split(", ")
        df.loc[index, 'Location']  = aux[0]
        df.loc[index, 'Location2'] = aux[1]

In [ ]:
countries = pd.DataFrame(columns=['country', 'occurrences'])

In [ ]:
for index, row in df.iterrows():
    if row['Location'] not in countries['country'].values and row['Location'] != '':
        countries = pd.concat([countries, pd.DataFrame([[row['Location'],int(1)]], columns=['country','occurrences'])])
    else:
        mask = countries['country'] == row['Location'] 
        countries.loc[mask, 'occurrences'] += 1
    if row['Location2'] not in countries['country'].values and row['Location2'] != '':
        countries = pd.concat([countries, pd.DataFrame([[row['Location2'],int(1)]], columns=['country','occurrences'])])
    else:
        mask = countries['country'] == row['Location2'] 
        countries.loc[mask, 'occurrences'] += 1

countries.reset_index(drop=True, inplace=True)  

In [ ]:
# ------------------------------------------------------------
# 2. Load world countries shapes
# ------------------------------------------------------------
NE_SAVED = 'Datasets/ne_10m_admin_0_countries.zip'
world = gpd.read_file(NE_SAVED)
world = world[world['NAME'] != 'Antarctica']

In [ ]:
# ------------------------------------------------------------
# 3. Correct countries names
# ------------------------------------------------------------

# Helper name mapping
def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode()
    return re.sub(r"[^a-z0-9]+", " ", s.lower()).strip()

def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode()
    return re.sub(r"[^a-z0-9]+", " ", s.lower()).strip()

synonyms = {
    "usa": "United States of America",
    "uk":  "United Kingdom",
    "uae": "United Arab Emirates",
    # other aliases
}

# mapping -> canonic name
names_norm = {_norm(n): n for n in world["NAME"]}

def map_name(raw: str) -> str | None:
    n = _norm(raw)
    if n in synonyms:
        return synonyms[n]
    if n in names_norm:
        return names_norm[n]
    m = difflib.get_close_matches(n, names_norm.keys(), n=1, cutoff=0.75)
    return names_norm[m[0]] if m else None

In [ ]:
# ------------------------------------------------------------
# 4. Join "countries" dataframe with "World" data
# ------------------------------------------------------------
countries["NAME"] = countries["country"].apply(map_name)
countries = countries.dropna(subset=["NAME"])
countries_unique = (countries.groupby("NAME", as_index=False, sort=False)["occurrences"].sum())
world = world.merge(countries_unique, on="NAME", how="left", validate="1:1")
world["occurrences"] = pd.to_numeric(world["occurrences"], errors="coerce")
world.loc[world["occurrences"].isna(), "occurrences"] = 0.0

In [ ]:
# ------------------------------------------------------------
# 5.  Make Europe a single polygon
# ------------------------------------------------------------
eu_exclude = {
    "Russia",
    "Turkey",
    "Kazakhstan",
    "Azerbaijan",
    "Georgia",
    "Armenia"
}

mask_eu = (world["CONTINENT"] == "Europe") & (~world["NAME"].isin(eu_exclude))
eu = (world[mask_eu].dissolve(by=lambda _: "Europe", aggfunc={"occurrences": "sum"}).reset_index().assign(NAME="Europe", CONTINENT="Europe"))
world_no_eu = world[~mask_eu].copy()
world_mixed = pd.concat([eu, world_no_eu], ignore_index=True)

In [ ]:
# ------------------------------------------------------------
# 6. Projection on the 0° meridian
# ------------------------------------------------------------
to_merc  = Transformer.from_crs("EPSG:4326", "+proj=merc +lon_0=0 +k=1 +datum=WGS84 +units=m +no_defs", always_xy=True).transform
to_equal = Transformer.from_crs("EPSG:4326", "+proj=moll +lon_0=0 +datum=WGS84 +units=m +no_defs", always_xy=True).transform
clip_box = box(-180, -85, 180, 85)
AREA_MIN = 5e9
world_patches    = []
world_patch_vals = []
world_label_xy   = []
world_label_vals = []

for geom, val in zip(world_mixed.geometry, world_mixed.occurrences):
    geom = geom.intersection(clip_box)
    if geom.is_empty:
        continue

    merc_geom   = transform(to_merc, geom)
    equal_geom  = transform(to_equal, geom)

    polys_merc  = merc_geom.geoms  if isinstance(merc_geom,  MultiPolygon) else [merc_geom]
    polys_equal = equal_geom.geoms if isinstance(equal_geom, MultiPolygon) else [equal_geom]

    keep_any = False
    
    for poly_m, poly_e in zip(polys_merc, polys_equal):
        if poly_e.area < AREA_MIN:
            continue
        keep_any = True
        world_patches.append(mplPolygon(np.asarray(poly_m.exterior.coords), closed=True))
        world_patch_vals.append(val)

    if keep_any and val:
        main_poly = max(polys_merc, key=lambda p: p.area)
        c = main_poly.centroid
        world_label_xy.append((c.x, c.y))
        world_label_vals.append(int(val))

In [ ]:
# ------------------------------------------------------------
# 7. Preapare the world plot
# ------------------------------------------------------------
def prepare_world_plot(patches, patch_vals):
    vals_log = np.log10(np.asarray(patch_vals) + 1)
    
    fig, ax = mplPlt.subplots(figsize=(15, 8))
    mplPlt.close(fig)
    
    colors = mplColors.LinearSegmentedColormap.from_list("", ["#ffffff", "#A467B5"], N=16384)
    
    pc = mplPC(patches, cmap=colors, edgecolor="#666666", linewidth=0.25)
    pc.set_array(vals_log)
    pc.set_clim(0, vals_log.max() or 1)
    
    all_x = [v for poly in patches for v in poly.get_xy()[:, 0]]
    all_y = [v for poly in patches for v in poly.get_xy()[:, 1]]
    
    lim_pad_x = (max(all_x) - min(all_x)) * 0.02
    lim_pad_y = (max(all_y) - min(all_y)) * 0.02
    
    return fig, ax, pc, (all_x, all_y, lim_pad_x, lim_pad_y)

In [ ]:
def contrast(rgb):
    r, g, b = rgb[:3]
    lum = 0.299*r + 0.587*g + 0.114*b
    return "black" if lum > 0.55 else "white"

In [ ]:
# ------------------------------------------------------------
# 8. Render the plot
# ------------------------------------------------------------
def render_world_plot(fig, ax, pc, limits_data, label_xy, label_vals, circles=False):
    all_x, all_y, lim_pad_x, lim_pad_y = limits_data
    
    ax.add_collection(pc)
    ax.set_xlim(min(all_x) - lim_pad_x, max(all_x) + lim_pad_x)
    ax.set_ylim(min(all_y) - lim_pad_y, max(all_y) + lim_pad_y)
    ax.set_aspect("equal")
    ax.axis("off")

    cmap = pc.cmap
    vmin, vmax = pc.get_clim()
    norm = mplColors.Normalize(vmin=vmin, vmax=vmax)

    for (x, y), v in zip(label_xy, label_vals):
        val_log   = np.log10(v + 1)
        shade_rgb = cmap(norm(val_log))
        txt_color = contrast(shade_rgb)

        if circles:
            if v < 10:
                halo_size_pts2 = 200
            else:
                halo_size_pts2 = 400
        
            if txt_color == "black":
                ax.text(x, y, str(v), ha="center", va="center", fontsize=8, color=txt_color, zorder=3)
                ax.scatter(x, y, s=halo_size_pts2, c="#dddddd", alpha=0.3, linewidths=0, zorder=2)
            else: 
                ax.text(x, y, str(v), ha="center", va="center", fontsize=8, color=txt_color, zorder=3, fontweight='bold')
                ax.scatter(x, y, s=halo_size_pts2, c="white", alpha=0.3, linewidths=0, zorder=2)

In [ ]:
# ------------------------------------------------------------
# 9. Animate the colouring of the countries
# ------------------------------------------------------------
def world_animation():
    fig, ax, pc, limits_data = prepare_world_plot(world_patches, world_patch_vals)

    vals_log_full = np.log10(np.asarray(world_patch_vals) + 1)
    vals_log_work = np.zeros_like(vals_log_full)

    pc.set_array(vals_log_work)
    pc.set_clim(0, vals_log_full.max())

    render_world_plot(fig, ax, pc, limits_data, label_xy=[], label_vals=[], circles=False)

    def update(frame):
        vals_log_work[frame] = vals_log_full[frame]
        pc.set_array(vals_log_work)
        return (pc,)

    frames_total = len(vals_log_full)

    return mplAnimation.FuncAnimation(fig, update, frames=frames_total, blit=True, repeat=False)

In [ ]:
world_anim = world_animation()
world_anim.save("Media/qc_organizations_world.mp4", writer = mplAnimation.FFMpegWriter(fps=60, codec="libx264"), dpi=200)

In [ ]:
# ------------------------------------------------------------
# 10. Make a map with colored countries and their stats
# ------------------------------------------------------------
def world_png():
    fig, ax, pc, limits_data = prepare_world_plot(world_patches, world_patch_vals)
    render_world_plot(fig, ax, pc, limits_data, label_xy=world_label_xy, label_vals=world_label_vals, circles=True)
    return fig

In [ ]:
world_fig = world_png()
world_fig.tight_layout()
world_fig.savefig("Media/qc_organizations_world.png", dpi=600)

In [30]:
# ------------------------------------------------------------
# 11. Make an histogram of the organizations on each country
# ------------------------------------------------------------
def world_hist():
    countries_min5 = countries.loc[countries['occurrences'] >= 5].sort_values('occurrences', ascending=False)
    total_bars = len(countries_min5)

    cmap = mplColors.LinearSegmentedColormap.from_list("", ["#A467B5", "#ffffff"], N=16384)
    colors = cmap(np.log1p(25*np.linspace(0,1,total_bars))/np.log1p(25))

    fig, ax = mplPlt.subplots(figsize=(10, 9))
    mplPlt.close(fig)
    
    bars = ax.bar(range(len(countries_min5)), np.zeros(len(countries_min5)), color=colors, edgecolor="black")
    
    def init_bars():
        for bar in bars:
            bar.set_height(0)
        return bars

    ax.set_xticks(range(len(countries_min5)))
    ax.set_xticklabels(countries_min5['country'], rotation=90)
    ax.set_ylim(0, countries_min5['occurrences'].max() * 1.1)
    ax.set_title("Organizzazioni di Quantum-computing per Nazione", fontsize=16, pad=30)
    ax.set_ylabel("Numero di Organizazioni", fontsize=16, labelpad=30)
    ax.set_xlabel("Nazione", fontsize=16, labelpad=30)

    steps_per_bar = 1
    total_frames = int((total_bars * steps_per_bar) * 1.1)
    final_heights = countries_min5['occurrences'].values

    def animate(frame):
        bar_idx = frame // steps_per_bar
        prog = (frame % steps_per_bar) / steps_per_bar

        for i, bar in enumerate(bars):
            if i < bar_idx:
                bar.set_height(final_heights[i])
            elif i == bar_idx:
                bar.set_height(final_heights[i] * prog)
            else:
                bar.set_height(0)
        return bars

    return mplAnimation.FuncAnimation(fig, animate, init_func=init_bars, frames=total_frames, blit=True)

In [31]:
hist_anim = world_hist()
hist_anim.save("Media/qc_organizations_histogram.mp4", writer=mplAnimation.FFMpegWriter(fps=20, codec="libx264"), dpi=200)